In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the ap-southeast-2 region. You will use the 544295431143.dkr.ecr.ap-southeast-2.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [107]:

bucket_name = 'firewhere1015'
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    else: 
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [87]:
# try:
#   urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
#   print('Success: downloaded bank_clean.csv.')
# except Exception as e:
#   print('Data load error: ',e)

try:
  model_data = pd.read_csv('./weatherAUS.csv')
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.


In [88]:
import pandas as pd


In [89]:
model_data.head(10)

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,No,0.0,No
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,No,0.0,No
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,No,0.0,No
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,No,1.0,No
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,No,0.2,No
5,14.6,29.7,0.2,56.0,19.0,24.0,55.0,23.0,1009.2,1005.4,20.6,28.9,No,0.0,No
6,14.3,25.0,0.0,50.0,20.0,24.0,49.0,19.0,1009.6,1008.2,18.1,24.6,No,0.0,No
7,7.7,26.7,0.0,35.0,6.0,17.0,48.0,19.0,1013.4,1010.1,16.3,25.5,No,0.0,No
8,9.7,31.9,0.0,80.0,7.0,28.0,42.0,9.0,1008.9,1003.6,18.3,30.2,No,1.4,Yes
9,13.1,30.1,1.4,28.0,15.0,11.0,58.0,27.0,1007.0,1005.7,20.1,28.2,Yes,0.0,No


In [90]:
model_data=model_data.fillna(0)

In [91]:
model_data.isnull().sum()

MinTemp          0
MaxTemp          0
Rainfall         0
WindGustSpeed    0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Temp9am          0
Temp3pm          0
RainToday        0
RISK_MM          0
RainTomorrow     0
dtype: int64

In [92]:
test = model_data

In [94]:
test["RainToday"]=model_data.RainToday.map(dict(Yes=1, No=0))

In [95]:
test["RainTomorrow"]=model_data.RainTomorrow .map(dict(Yes=1, No=0))

In [96]:
test.head(10)

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,0.0,0.0,0
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,0.0,0.0,0
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,0.0,0.0,0
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,0.0,1.0,0
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,0.0,0.2,0
5,14.6,29.7,0.2,56.0,19.0,24.0,55.0,23.0,1009.2,1005.4,20.6,28.9,0.0,0.0,0
6,14.3,25.0,0.0,50.0,20.0,24.0,49.0,19.0,1009.6,1008.2,18.1,24.6,0.0,0.0,0
7,7.7,26.7,0.0,35.0,6.0,17.0,48.0,19.0,1013.4,1010.1,16.3,25.5,0.0,0.0,0
8,9.7,31.9,0.0,80.0,7.0,28.0,42.0,9.0,1008.9,1003.6,18.3,30.2,0.0,1.4,1
9,13.1,30.1,1.4,28.0,15.0,11.0,58.0,27.0,1007.0,1005.7,20.1,28.2,1.0,0.0,0


In [82]:
#test = test.drop(['RainTomorrow'],axis =1)

In [97]:
model_data= test

In [98]:
model_data.head(10)

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,0.0,0.0,0
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,0.0,0.0,0
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,0.0,0.0,0
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,0.0,1.0,0
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,0.0,0.2,0
5,14.6,29.7,0.2,56.0,19.0,24.0,55.0,23.0,1009.2,1005.4,20.6,28.9,0.0,0.0,0
6,14.3,25.0,0.0,50.0,20.0,24.0,49.0,19.0,1009.6,1008.2,18.1,24.6,0.0,0.0,0
7,7.7,26.7,0.0,35.0,6.0,17.0,48.0,19.0,1013.4,1010.1,16.3,25.5,0.0,0.0,0
8,9.7,31.9,0.0,80.0,7.0,28.0,42.0,9.0,1008.9,1003.6,18.3,30.2,0.0,1.4,1
9,13.1,30.1,1.4,28.0,15.0,11.0,58.0,27.0,1007.0,1005.7,20.1,28.2,1.0,0.0,0


In [99]:
import numpy as np

In [100]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1727), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(99535, 15) (42658, 15)


In [112]:
train_data.head(10)

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
42478,17.4,24.1,4.4,33.0,11.0,15.0,79.0,73.0,1017.5,1016.6,21.1,23.5,1.0,0.0,0
114931,12.2,17.9,1.8,50.0,24.0,20.0,73.0,72.0,1011.5,1010.3,16.0,15.8,1.0,1.2,1
141493,5.2,16.7,0.0,39.0,13.0,15.0,75.0,39.0,1026.8,1024.0,10.5,16.0,0.0,0.0,0
26377,17.6,28.7,0.0,43.0,9.0,15.0,62.0,46.0,0.0,0.0,22.0,26.9,0.0,2.4,1
29589,10.9,19.6,0.0,0.0,19.0,11.0,62.0,52.0,1019.3,1016.9,14.1,18.6,0.0,0.0,0
65324,13.6,16.6,6.6,76.0,33.0,35.0,54.0,47.0,1007.0,1011.3,15.1,15.3,1.0,0.0,0
56027,3.7,9.9,0.0,28.0,13.0,19.0,76.0,65.0,1026.8,1024.5,5.7,8.3,0.0,0.0,0
690,13.8,18.7,50.8,52.0,17.0,22.0,78.0,67.0,1011.3,1012.0,15.5,17.6,1.0,1.2,1
68446,8.5,21.1,0.2,41.0,19.0,17.0,70.0,36.0,1017.0,1013.1,13.8,20.5,0.0,0.0,0
139712,21.8,39.0,0.0,44.0,11.0,22.0,43.0,48.0,1014.7,1008.8,30.6,36.0,0.0,0.0,0


In [108]:
pd.concat([train_data['RainTomorrow'], train_data.drop(['RainTomorrow'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')


In [109]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1, instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [110]:
xgb.fit({'train': s3_input_train})

2021-10-16 14:32:28 Starting - Starting the training job...
2021-10-16 14:32:51 Starting - Launching requested ML instancesProfilerReport-1634394747: InProgress
......
2021-10-16 14:33:52 Starting - Preparing the instances for training............
2021-10-16 14:35:52 Downloading - Downloading input data
2021-10-16 14:35:52 Training - Downloading the training image..Arguments: train
[2021-10-16:14:36:11:INFO] Running standalone xgboost training.
[2021-10-16:14:36:11:INFO] Path /opt/ml/input/data/validation does not exist!
[2021-10-16:14:36:11:INFO] File size need to be processed in the node: 6.77mb. Available memory size in the node: 8404.95mb
[2021-10-16:14:36:11:INFO] Determined delimiter of CSV input is ','
[14:36:11] S3DistributionType set as FullyReplicated
[14:36:11] 99535x14 matrix with 1393490 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[14:36:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_de

In [111]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

-----!